In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

In [2]:
symbols=pd.read_excel('data_final.xlsx')['symbol']

# Probabilidad de transicción en cada estado 

$$p_{ij}=\frac{n_{ij}}{\sum_{j}n_{ij}}$$

### Estados

$$r \in S_{1} \rightarrow  r\leq C_{1}$$

$$r \in S_{2} \rightarrow  C_{1}<r\leq C_{2}$$

$$r \in  S_{3} \rightarrow  C_{2}<r\leq C_{3}$$

$$r \in S_{4} \rightarrow  C_{3}<r$$


In [3]:
def cadena_markov_t(symbol):
    def rendimientos(com):return np.diff(np.log(com))*100
    cierre=list(yf.download(symbol,start='2019-1-1',progress=0)['Close'])
    returns=rendimientos(cierre)
    c1,c2,c3=np.quantile(returns,[0.25,0.5,0.75])
    def status_rend(rendimiento):return "1" if rendimiento<=c1 else "2" if rendimiento<=c2 else "3" if rendimiento<=c3 else "4"
    stat_ren=list(map(status_rend,returns))
    cambios=[stat_ren[i]+stat_ren[i+1] for i in range(len(stat_ren)-1)]
    num_estados=[stat_ren.count(str(i)) for i in range(1,5)]
    matrix=[cambios.count(str(i)+str(j))/num_estados[i-1]  if num_estados[i-1]>0 else 0 for i in range(1,5) for j in range(1,5)]
    matrix=[round(m,2) for m in matrix]
    cadena_markov=pd.DataFrame(np.array(matrix).reshape(4,4),columns=[1,2,3,4],index=[1,2,3,4])
    cadena_markov.index.name='Estado Inicial'
    cadena_markov.columns.name='Estado Final'
    last_status_rend=int(stat_ren[-1])-1
    vector=list(np.array(cadena_markov)[last_status_rend])
    return cadena_markov,vector.index(max(vector))+1,last_status_rend+1

In [4]:
matrices=list(map(cadena_markov_t,symbols))

In [4]:
cadena_markov_t(symbols[0])

(Estado Final       1     2     3     4
 Estado Inicial                        
 1               0.25  0.25  0.18  0.31
 2               0.23  0.23  0.27  0.27
 3               0.24  0.28  0.29  0.18
 4               0.27  0.24  0.25  0.24, 3, 3)

In [5]:
# Matriz de transición de cada estado, 
#estado del ultimo rendimiento, estado con mayor prob de transicción
matrices[78][0]

Estado Final,1,2,3,4
Estado Inicial,,,,
1,0.22,0.25,0.24,0.29
2,0.22,0.24,0.31,0.23
3,0.22,0.29,0.25,0.24
4,0.34,0.22,0.20,0.24


In [6]:
matrices

[(Estado Final       1     2     3     4
  Estado Inicial                        
  1               0.26  0.26  0.18  0.31
  2               0.23  0.22  0.28  0.27
  3               0.25  0.28  0.29  0.18
  4               0.26  0.24  0.25  0.24, 1),
 (Estado Final       1     2     3     4
  Estado Inicial                        
  1               0.28  0.20  0.21  0.31
  2               0.22  0.28  0.25  0.24
  3               0.19  0.30  0.29  0.22
  4               0.31  0.21  0.25  0.23, 1),
 (Estado Final       1     2     3     4
  Estado Inicial                        
  1               0.27  0.27  0.23  0.23
  2               0.25  0.28  0.23  0.25
  3               0.21  0.22  0.33  0.24
  4               0.27  0.23  0.21  0.29, 4),
 (Estado Final       1     2     3     4
  Estado Inicial                        
  1               0.25  0.22  0.21  0.32
  2               0.20  0.34  0.20  0.26
  3               0.27  0.25  0.26  0.22
  4               0.28  0.27  0.26  0.19, 